In [10]:
from PIL import Image
from collections import Counter, defaultdict
import heapq
import numpy as np


class HuffmanNode:
    """Класс для узлов дерева Хаффмана"""
    def __init__(self, value, freq):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq


def build_huffman_tree(frequencies):
    """Построение дерева Хаффмана на основе частот пикселей"""
    heap = [HuffmanNode(value, freq) for value, freq in frequencies.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)

        merged = HuffmanNode(None, node1.freq + node2.freq)
        merged.left = node1
        merged.right = node2

        heapq.heappush(heap, merged)

    return heap[0]  # Корень дерева


def build_huffman_codes(tree):
    """Создание таблицы кодов Хаффмана"""
    codes = {}

    def generate_codes(node, current_code):
        if node is None:
            return
        if node.value is not None:
            codes[node.value] = current_code
            return
        generate_codes(node.left, current_code + "0")
        generate_codes(node.right, current_code + "1")

    generate_codes(tree, "")
    return codes


def compress_image(image_array, codes):
    """Сжатие изображения с использованием кодов Хаффмана"""
    compressed_data = "".join(codes[pixel] for pixel in image_array.flatten())
    return compressed_data


def decompress_image(compressed_data, tree, shape):
    """Восстановление изображения из сжатого состояния"""
    decompressed_data = []
    node = tree

    for bit in compressed_data:
        if bit == "0":
            node = node.left
        else:
            node = node.right

        if node.left is None and node.right is None:
            decompressed_data.append(node.value)
            node = tree

    return np.array(decompressed_data).reshape(shape)


def main():
    # Шаг 1: Загрузка изображения
    image_path = "image.bmp"  # Путь к изображению
    image = Image.open(image_path).convert("L")  # Конвертация в градации серого
    image_array = np.array(image)

    # Шаг 2: Построение дерева Хаффмана
    pixel_frequencies = Counter(image_array.flatten())
    huffman_tree = build_huffman_tree(pixel_frequencies)
    huffman_codes = build_huffman_codes(huffman_tree)

    print("Коды Хаффмана для каждого пикселя:")
    for pixel, code in huffman_codes.items():
        print(f"{pixel}: {code}")

    # Шаг 3: Сжатие изображения
    compressed_data = compress_image(image_array, huffman_codes)
    print(f"Сжатые данные: {compressed_data[:100]}... (первые 100 бит)")

    # Шаг 4: Восстановление изображения
    decompressed_array = decompress_image(compressed_data, huffman_tree, image_array.shape)
    decompressed_image = Image.fromarray(decompressed_array.astype(np.uint8))

    # Сохранение восстановленного изображения
    decompressed_image.save("decompressed_image.bmp")
    print("Восстановленное изображение сохранено как 'decompressed_image.bmp'.")


if __name__ == "__main__":
    main()


Коды Хаффмана для каждого пикселя:
192: 00
251: 01000
64: 01001
154: 0101
164: 0110
208: 0111
160: 100
135: 10100000
109: 101000010
121: 101000011
166: 1010001
38: 1010010
102: 10100110
74: 10100111
156: 10101
126: 10110
83: 101110
90: 10111100
196: 101111010
45: 1011110110
130: 10111101110000
71: 101111011100010
26: 101111011100011
255: 1011110111001
111: 1011110111010
47: 1011110111011
57: 101111011110
175: 1011110111110000
66: 10111101111100010
194: 101111011111000110
137: 1011110111110001110
97: 10111101111100011110
81: 10111101111100011111
55: 101111011111001
119: 10111101111101
173: 1011110111111
185: 1011111
128: 1100
182: 11010
0: 11011
92: 11100
116: 11101
145: 1111000
147: 1111001
138: 11110100
54: 11110101
36: 1111011
118: 111110
100: 111111
Сжатые данные: 0101010101010101010101010101010101010101010101010101010101010101010101010101010101010110011001100110... (первые 100 бит)
Восстановленное изображение сохранено как 'decompressed_image.bmp'.
